In [1]:
from numpy import *

## 从文本中构建词向量

In [2]:
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

In [3]:
def createVocabList(dataSet):         
    '''创建词汇表，将数据集转化为一个没有重复词汇的词汇表'''
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

In [4]:
def setOfWord2Vec(vocabList,inputSet):
    '''将文档转化def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec为向量'''
    returnVec = [0]*len(vocabList)                                      #根据词袋的长度确定词向量的长度
    for word in inputSet:                                               #若输入数据中的词存在于词袋中，则词向量的相应位置置1
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print('the word:{} is not in my vocabulary!'.format(word))
    return returnVec               

In [5]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [6]:
#listOPosts,listClasses = loadDataSet()
#myVocabList = createVocabList(listOPosts)
#print(myVocabList)
#wordVec = setOfWord2Vec(myVocabList,listOPosts[0])
#print(wordVec)

In [7]:
def train_bayes(trainMatrix,trainCategory):
    '''训练贝叶斯分类器：输入为文档矩阵和类别标签向量'''
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)                #计算文档中侮辱性文档所占的概率
    
    p0Num = ones(numWords);p1Num = ones(numWords)
    p0Denom = 2.0;p1Denom = 2.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom +=sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom +=sum(trainMatrix[i])
        p1Vect = log(p1Num/p1Denom)
        p0Vect = log(p0Num/p0Denom)
        return p0Vect,p1Vect,pAbusive

In [8]:
def classify(vec2Classify,p0Vec,p1Vec,pClass1):
    ''''''
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1.0-pClass1)
    if p1>=p0:
        return 1
    else:
        return 0


In [9]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
trainMat=[]
for postinDoc in listOPosts:
    trainMat.append(setOfWord2Vec(myVocabList, postinDoc))
    
p0V,p1V,pAb = train_bayes(array(trainMat),array(listClasses))


testEntry = ['love', 'my', 'dalmation']
thisDoc = array(setOfWord2Vec(myVocabList, testEntry))
print(thisDoc)
print(testEntry,'classified as: ',classify(thisDoc,p0V,p1V,pAb))

testEntry = ['stupid', 'garbage']
thisDoc = array(setOfWord2Vec(myVocabList, testEntry))
print(testEntry,'classified as: ',classify(thisDoc,p0V,p1V,pAb))

[0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['love', 'my', 'dalmation'] classified as:  1
['stupid', 'garbage'] classified as:  1


In [10]:
def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 
    
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50)); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = train_bayes(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classify(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error",docList[docIndex])
    print('the error rate is: ',float(errorCount)/len(testSet))
    #return vocabList,fullText

In [11]:
spamTest()

classification error ['ordercializviagra', 'online', 'save', '0nline', 'pharmacy', 'noprescription', 'required', 'buy', 'canadian', 'drugs', 'wholesale', 'prices', 'and', 'save', 'fda', 'approved', 'drugs', 'superb', 'quality', 'drugs', 'only', 'accept', 'all', 'major', 'credit', 'cards']
classification error ['percocet', '625', 'withoutprescription', 'tabs', '225', 'percocet', 'narcotic', 'analgesic', 'used', 'treat', 'moderate', 'moderately', 'severepain', 'top', 'quality', 'express', 'shipping', '100', 'safe', 'discreet', 'private', 'buy', 'cheap', 'percocet', 'online']
classification error ['you', 'have', 'everything', 'gain', 'incredib1e', 'gains', 'length', 'inches', 'yourpenis', 'permanantly', 'amazing', 'increase', 'thickness', 'yourpenis', 'betterejacu1ation', 'control', 'experience', 'rock', 'harderecetions', 'explosive', 'intenseorgasns', 'increase', 'volume', 'ofejacu1ate', 'doctor', 'designed', 'and', 'endorsed', '100', 'herbal', '100', 'natural', '100', 'safe', 'the', 'pr

D:\Anaconda3\envs\ame\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
